In [7]:
#Import Dependencies
import csv, sqlite3
import pandas as pd

In [8]:
#DATABASE name
database = '../db_testing/project3db.sqlite'

In [9]:
db = sqlite3.connect(database)
cursor = db.cursor()
res = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print (name[0])
db.close()

Alc_test1
sqlite_sequence
HA_test_phone1
HA_test_watch1


In [10]:
#connect to the database
db = sqlite3.connect(database)

query = "SELECT * FROM HA_test_phone1;"
df = pd.read_sql_query(query,db)
db.close()
df.head()

,rec_id,x,y,z,Status
0,1,-1.725723,-0.713531,5.246567,bike
1,2,-6.947235,0.870163,12.201019,walk
2,3,-2.430313,-2.883240,11.265335,walk
3,4,5.292801,0.265991,8.349380,sit
4,5,-2.705246,1.418091,9.863297,bike


In [11]:
y_test = df["Status"]
y_test

0            bike
1            walk
2            walk
3             sit
4            bike
5           stand
6             sit
7        stairsup
8        stairsup
9            walk
10          stand
11       stairsup
12            sit
13           bike
14           bike
15       stairsup
16           bike
17            sit
18     stairsdown
19          stand
20          stand
21          stand
22          stand
23            sit
24            sit
25           walk
26           walk
27          stand
28           bike
29       stairsup
          ...    
970          bike
971      stairsup
972      stairsup
973         stand
974           sit
975          walk
976           sit
977          walk
978          bike
979         stand
980          walk
981          bike
982           sit
983         stand
984    stairsdown
985    stairsdown
986    stairsdown
987    stairsdown
988          bike
989      stairsup
990      stairsup
991      stairsup
992      stairsup
993      stairsup
994       

In [13]:
X_test = df.iloc[:,1:4]
X_test.head()

,x,y,z
0,-1.725723,-0.713531,5.246567
1,-6.947235,0.870163,12.201019
2,-2.430313,-2.883240,11.265335
3,5.292801,0.265991,8.349380
4,-2.705246,1.418091,9.863297


In [14]:
#Load the Scaled function for LR model
from sklearn.externals.joblib import dump, load
X_scaler = load('HAP_scaler.bin')

In [15]:
#Scales X_test data
X_test_scaled = X_scaler.transform(X_test)  

In [16]:
#Load Label Encoder
label_encoder = load('HAP_label_encoder.bin')

In [17]:
from keras.utils import to_categorical
encoded_y_test = label_encoder.transform(y_test)
y_test_categorical = to_categorical(encoded_y_test)

Using TensorFlow backend.


In [19]:
import numpy as np
X_test_scaled = np.reshape(X_test_scaled, (X_test_scaled.shape[0],1,X_test_scaled.shape[1]))
X_test_scaled = np.array(X_test_scaled)

In [21]:
#Import a model (NN)
from keras.models import load_model
modeln1 = "HA_RNN1_PhoneT.h5"
model = load_model(modeln1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [22]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.29502049684524534, Accuracy: 0.8705000281333923


In [26]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [27]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: ['walk' 'walk' 'bike' 'sit' 'bike' 'stand' 'sit' 'sit' 'bike' 'bike']
Actual Labels: ['bike', 'walk', 'walk', 'sit', 'bike', 'stand', 'sit', 'stairsup', 'stairsup', 'walk']
